
#**FDI Detection Data Generator New**

Importing Libraries

In [ ]:
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.stats import multivariate_normal

mpl.rcParams['legend.fontsize'] = 10

Defining Sensors Parameters

In [ ]:
n_sensors = 2

#A=np.random.rand(2,2)
#for i in range(0,A.shape[0]):
#    A[:,i]=0.5*A[:,i]/np.sum(A[:,i])
A = np.array([[1,0],[0,1]])
C = []
C.append(np.array([[0.5,0],[0,1]]))
C.append(np.array([[0,0.5],[1,0]]))
C = np.array(C)
C = np.reshape(C, (C.shape[0]*C.shape[1], C.shape[2]))

Q = np.identity(2)
mu_x = np.zeros((2,))

R = np.identity(4)
mu_y = np.zeros((4,))

Defining Estimation Functions

In [ ]:
def random(mu, var):
	return np.random.multivariate_normal(mu, var)

def process(x):
	x_new = np.dot(A,x) + random(mu_x, Q)
	return x_new

def observation(x):
	y_new = np.dot(C,x) + random(mu_y, R)
	return y_new

def run_process(T):      #T is time
	x=[]
	x.append(np.zeros(2,))
	y_start=np.dot(C,x[0]) + random(mu_y, R)
	y=[y_start]
	for j in range(T-1):
		x_temp = process(x[j])
		x.append(x_temp)
		y.append(observation(x_temp))
	return (np.array(x).T,np.array(y).T)



def estimate(z_k, x_k, P_k):  #
	I = np.identity(2)
	P_k_n = A @ P_k @ A.T + Q
	K_k = P_k_n @ C.T @ np.linalg.inv(C @ P_k_n @ C.T + R)
	x_k_n = np.dot(A, x_k) + np.dot(K_k, z_k)
	P_k = (I - K_k @ C) @ P_k_n
	return (P_k, x_k_n)

def detect(x_hat, y_k):
	z_k = y_k - C @ A @ x_hat
	return z_k

def attack(z_k, T, sigma_b):
	return np.dot(T,z_k) + random(np.zeros((2,)),sigma_b)


def run(y, attack_instant, T, sigma_b):
	P_0 = np.identity(2)
	P_k = P_0
	x_hat = []
	z = []
	P = []
	#T = np.identity(2)
	#sigma_b = np.identity(2)

	x_hat_k = np.zeros(2,)
	for k in range(N):
		z_k = detect(x_hat_k, y[:,k])
		if(k>=attack_instant and attack_instant != -1):
			z_k[:2] = attack(z_k[:2], T, sigma_b)
		z.append(z_k)
		(P_k, x_hat_k) = estimate(z_k, x_hat_k, P_k)
		x_hat.append(x_hat_k)
		P.append(P_k)
	var = C @ (A @ P_k @ A.T + Q) @ C.T + R
	return (np.array(x_hat).T, np.array(z).T, P, var)

def estimate_after_attack(z_k, x_k, P_k, R_s, R_a):
	I = np.identity(2)
	R_ = np.copy(R)
	R_[:2,:2] = R_a
	R_[2:,2:] = R_s
	P_k_n = A @ P_k @ A.T + Q
	K_k = P_k_n @ C.T @ np.linalg.inv(C @ P_k_n @ C.T + R_)
	x_k_n = np.dot(A, x_k) + np.dot(K_k, z_k)
	P_k = (I - K_k @ C) @ P_k_n
	return (P_k, x_k_n)

def run_attack(y, attack_instant, T, sigma_b, x_hat, z, P, len):
	P_0 = P[attack_instant-1]
	P_k = P_0
	x_hat_corr = []
	R_s = R[2:,2:]
	T_inv = np.linalg.inv(T)
	R_a = R[:2,:2] + T_inv @ sigma_b @ T_inv.T
	R_ = np.copy(R)
	R_[:2,:2] = R_a
	T_ = np.identity(4)
	T_[:2,:2] = T
	mean = np.copy(z[:,attack_instant:])
	#var_s = []
	#var_a = []
	var = []
	z_ret = np.copy(z[:,attack_instant:])
	C_a = C[:2,:]
	C_s = C[2:,:]
	x_hat_k = x_hat[:,attack_instant - 1]
	for k in range(attack_instant, len+1):
		P_ = A @ P_k @ A.T + Q
		#var_s.append(C_s @ P_ @ C_s.T + R_s)
		#var_a.append(T @ (C_a @ P_ @ C_a.T + R_a) @ T.T)
		var.append(T_ @ (C @ P_ @ C.T + R_) @ T_.T)
		z_ret[:2,k-attack_instant] = np.dot(np.linalg.inv(T),z[:2,k]) + np.dot(C_a @ A, x_hat[:,k-1] - x_hat_k)
		z_ret[2:,k-attack_instant] = z[2:,k] + np.dot(C_s @ A, x_hat[:,k-1] - x_hat_k)
		(P_k, x_hat_k) = estimate_after_attack(z_ret[:,k-attack_instant], x_hat_k, P_k, R_s, R_a)
		x_hat_corr.append(x_hat_k)
		mean[:2,k-attack_instant] = np.dot(T @ C_a @ A, x_hat_k - x_hat[:,k-1])
		mean[2:,k-attack_instant] = np.dot(C_s @ A, x_hat_k - x_hat[:,k-1])
	return (np.array(x_hat_corr).T, np.array(mean).T, var)
 
def probab_calculation(z, steady_var, y, x_hat, P, limit):
	Pmat = np.zeros((N,N))   ### row-> z_k  col->attack_instant
	for i in range(limit):  ##### This can be change to actual attack instant + some delay
		(x_hat_corr, mean, var_after_attack) = run_attack(y, i, T, sigma_b, x_hat, z, P, N-1)
		for j in range(limit):    ##### This can be change to actual attack instant + some delay
			if (j<i):
				Pmat[i,j] = multivariate_normal(mean=np.zeros((4,)), cov=np.array(steady_var)).pdf(z[:,j]) 
			else:
				Pmat[i,j] = multivariate_normal(mean=mean[j-i,:], cov=np.array(var_after_attack)[j-i,:,:]).pdf(z[:,j])
	
	Pmat_history = np.zeros((N,N))
	Pmat_history[:,0] = Pmat[:,0]
	for j in range(1,N):
		Pmat_history[:,j] = Pmat_history[:,j-1]*Pmat[:,j]
	return (Pmat, Pmat_history)


Defining Posterior Belief Probability Function

In [ ]:
def get_pi_seq(z,steady_var,y,x_hat,P, real_attack_instant):
    pi=[]
    pi.append(pi_0)
    (Pmat, Pmat_history) = probab_calculation(z, steady_var, y, x_hat, P, min(N,real_attack_instant + 10))
    for i in range(N):
        #print(str(i)+":  "+str(pi[i]))
        if pi[i]>0.99:
            print('0.99 surpassed at time', i)
            print('attack at time', attack_instant)
            for k in range(i,N):
                pi.append(1)
            break
        
        beta=pi[i]/((1-pi[i])*(1-theta))+theta/(1-theta)
        beta=beta/(multivariate_normal(mean=[0,0,0,0], cov=steady_var).pdf(z[:,i])+0.0001)
        if (i == real_attack_instant + 9):
            (Pmat, Pmat_history) = probab_calculation(z, steady_var, y, x_hat, P, N)

        pc_term=0
        pc_den = 0
        for j in range(i+1):  #Considering prior probab of attack =0
            P_term=((1-theta)**(j))*theta/(1-(1-theta)**(i+1))
            if (i>0):
                P_term*=Pmat_history[j,i-1]
            pc_den += P_term
            
            #(x_hat_corr, mean, var_after_attack) = run_attack(y,j, T, sigma_b, x_hat, z, P, i)
            #probab_z_term1 = multivariate_normal(mean=mean[i-j,:], cov=np.array(var_after_attack)[i-j,:,:]).pdf(z[:,i])
            probab_z_term = Pmat[j,i]
            #if(probab_z_term != probab_z_term1):
            #    print (str(probab_z_term)+"    "+str(probab_z_term1))
            #    print ("CAUTION: VALUES MISMATCH!!!")
            pc_term=pc_term+P_term*probab_z_term
        pc_term /= pc_den
        beta=beta*pc_term
        pi.append(beta/(1+beta))
        
    return pi

##**Generating Data**

Defining hyperparameters & variables for generating data

In [ ]:
N=100
theta=0.05  # https://homepage.divms.uiowa.edu/~mbognar/applets/geo1.html
#sets P(t>100)=0.00562


pi_0=0.0                #Prior probability of attack
T=np.eye(2)
b=np.ones(2)*0

z_set=[]
pi_set=[]
attack_instance_set=[]
y_set = []
x_set = []
x_hat_set = []
num_paths=10000

T = np.identity(2)*(-1)    #Attack Matrix T
sigma_b = np.identity(2)*0 #Variance of Attack noise

Running loop to generate sample paths

In [ ]:
for i in range(num_paths):
    t=np.random.geometric(p=theta)

    if t>N:
        t=-1
    attack_instant =t                  #change =-1 to test for false positives

    print('path', i, ':')
    (x,y) = run_process(N)
    (x_hat, z, P, var) = run(y, attack_instant, T, sigma_b)
    pi=get_pi_seq(z,var,y,x_hat,P,t)
    
    z_set.append(z)
    pi_set.append(pi)
    attack_instance_set.append(attack_instant)
    x_set.append(x)
    y_set.append(y)
    x_hat_set.append(x_hat)

Saving the data

In [ ]:
np.save('z_set.npy',z_set)
np.save('pi_set.npy',pi_set)
np.save('attack_instance_set.npy',attack_instance_set)
np.save('x_set.npy', x_set)
np.save('y_set.npy', y_set)
np.save('x_hat_set.npy', x_hat_set)

In [ ]:
print(A)

[[1 0]
 [0 1]]


In [ ]:
np.save('A.npy',A)